<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import queue
baseURL = 'https://www.soccerbase.com'
lock = threading.Lock()

In [18]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    lock.acquire()
    print(func.__name__, 'took', end = ' ')
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
    lock.release()
  return timeIt

In [19]:
@timer
def getEveryCompetitionData():
  request = requests.get("https://www.soccerbase.com/players/home.sd")
  soup = BeautifulSoup(request.content, "lxml")
  testing = soup.find('div', id = 'players-menu')
  for div in testing.find_all('div', class_ = 'soccerColumn'):
    if (re.search('Competition', div.text) != None):
      comp = div
  threads = []
  compQ = queue.Queue()
  for competition in comp.find_all('a'):
    t = threading.Thread(target = readCompetitionPage, args = (competition.text, baseURL + competition['href'], compQ))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  counter = 1
  while not compQ.empty():
    print(counter, compQ.get()[0])
    counter += 1

#From competition page of teams
## seasons: dataframe consisted of each season value with the season date

In [20]:
#From player page for a team
def getTeamData(team, url, q):
  names = [team]
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
  for elem in test.find_all('b'):
    name = elem.find('a')
    names.append(name.text)
  q.put(names)

In [21]:
import threading
import queue
@timer
def readCompetitionPage(compURL):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  seasons = season_dataframes(soup)
  teams = season_selector(seasons, "2019/20")
  print(teams)

In [22]:
def season_selector(seasons, date):
  value = seasons.loc[date]["value"]
  url = "https://www.soccerbase.com/tournaments/tournament.sd?tourn_id=" + value
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  teamNames = soup.find("table", class_ = "table")
  team_df = teams_dataframes(teamNames)
  #going to team url "https://www.soccerbase.com + team_df.loc[teamName]["link"]
  teams = {}
  threads = []
  q = queue.Queue()
  for index, row in team_df.iterrows():
    t = threading.Thread(target = getTeamData, args = (index, "https://www.soccerbase.com" + row["link"], q))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  while not q.empty():
    teamData = q.get()
    teams[teamData[0]] = teamData[1:]
  return teams

Dataframes
season_dataframes: return the dataframes consisted of seasons and each season values
teams_dataframes: return the dataframe consisted of team names and the link to read data of a team

In [23]:
import re
import pandas as pd
def season_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("select", id = "seasonSelect")
  for data in list.find_all("option")[1:]:
    df = df.append({"value": data["value"], "date" : data.text}, ignore_index = True)
  df.set_index("date", inplace = True)
  
  return df


In [24]:
def teams_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("tbody")
  for data in list.find_all("td", class_ = "bull"):
    team = data.find("a")
    df = df.append({"link": team["href"], "name" : team.text}, ignore_index = True)
  df.set_index("name", inplace = True)
  return df

In [25]:
#readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')
getEveryCompetitionData()

getEveryCompetitionData took 0.38 seconds to run.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-c7feaa8ba089>", line 6, in timeIt
    func(*args, **kwargs)
TypeError: readCompetitionPage() takes 1 positional argument but 3 were given

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-c7feaa8ba089>", line 6, in timeIt
    func(*args, **kwargs)
TypeError: readCompetitionPage() takes 1 positional argument but 3 were given

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  Fi